<a href="https://colab.research.google.com/github/gwangjinjeong/Pytorch/blob/master/basic/02_Dataset_Transform_Compose_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set Image Datasets and Transforms and Compose

## 0. Preparation

In [1]:
import torch
from torch.utils.data import Dataset
torch.manual_seed(1)

## 1. Define class for toy dataset


image dataset과 비슷하게, 설정해보자   
형식은 다음과 같다
* [0] : data (vector형태)
* [1] : class

In [0]:
class toy_set(Dataset):

    def __init__(self, length = 100, transform = None):
        self.len = length
        self.x = 2 * torch.ones(length, 2)
        self.y = torch.ones(length, 1)
        self.transform = transform
     
    # __getitem__ :  indexing 할때나, slicing할때, 아이템을 어떻게 얻을 것인지 설정할수 있다.
    def __getitem__(self, index):
        sample = self.x[index], self.y[index]
        if self.transform:
            sample = self.transform(sample)     
        return sample
    # __len__ : len을 호출할때 무엇을 반환시킬것인지 설정 가능하다.
    def __len__(self):
        return self.len

In [6]:
our_dataset = toy_set()
print(our_dataset[0])
print(len(our_dataset))

(tensor([2., 2.]), tensor([1.]))
100


In [7]:
for i in range(3):
    x, y=our_dataset[i]
    print("index: ", i, '; x:', x, '; y:', y)

index:  0 ; x: tensor([2., 2.]) ; y: tensor([1.])
index:  1 ; x: tensor([2., 2.]) ; y: tensor([1.])
index:  2 ; x: tensor([2., 2.]) ; y: tensor([1.])


##2. Transforms

In [0]:
class add_mult(object):
    
    def __init__(self, addx = 1, muly = 2):
        self.addx = addx
        self.muly = muly
    
    # class 호출시 즉시실행
    def __call__(self, sample):
        x = sample[0]
        y = sample[1]
        x = x + self.addx
        y = y * self.muly
        sample = x, y
        return sample

In [0]:
a_m = add_mult()
data_set = toy_set()

### 2.1 Dataset 단일적용

In [11]:
for i in range(3):
    x, y = data_set[i]
    print('Original')
    print(i,x,y)
    print('Transformed')
    x_, y_ = a_m(data_set[i])
    print(i, x_, y_)
    print('-'*30)

Original
0 tensor([2., 2.]) tensor([1.])
Transformed
0 tensor([3., 3.]) tensor([2.])
------------------------------
Original
1 tensor([2., 2.]) tensor([1.])
Transformed
1 tensor([3., 3.]) tensor([2.])
------------------------------
Original
2 tensor([2., 2.]) tensor([1.])
Transformed
2 tensor([3., 3.]) tensor([2.])
------------------------------


### 2.2 Dataset 전체적용

In [0]:
cust_data_set = toy_set(transform = a_m)

In [14]:
for i in range(3):
    x, y = data_set[i]
    print('Original')
    print(i,x,y)
    print('Transformed')
    x_, y_ = cust_data_set[i]
    print(i, x_, y_)
    print('-'*30)

Original
0 tensor([2., 2.]) tensor([1.])
Transformed
0 tensor([3., 3.]) tensor([2.])
------------------------------
Original
1 tensor([2., 2.]) tensor([1.])
Transformed
1 tensor([3., 3.]) tensor([2.])
------------------------------
Original
2 tensor([2., 2.]) tensor([1.])
Transformed
2 tensor([3., 3.]) tensor([2.])
------------------------------


## 3. Compose
Transform 여러개를 한개로 만들어주는것

In [0]:
try:
  from torchvision import transforms
except:
  !pip install -y torchvision

<img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0110EN/notebook_images%20/chapter%201/1.3.1_trasform.png" width="500" alt="Compose PyTorch">

In [0]:
class mult(object):

    def __init__(self, mult = 100):
        self.mult = mult
        
    def __call__(self, sample):
        x = sample[0]
        y = sample[1]
        x = x * self.mult
        y = y * self.mult
        sample = x, y
        return sample

transform add_mult와 mult를 하나로 처리한다.

### 3.1 Dataset 단일적용

In [0]:
data_transform = transforms.Compose([add_mult(), mult()])

In [19]:
data_transform(data_set[0])

(tensor([300., 300.]), tensor([200.]))

In [20]:
x,y=data_set[0]
x_,y_=data_transform(data_set[0])
print( 'Original x: ', x, 'Original y: ', y)
print( 'Transformed x_:', x_, 'Transformed y_:', y_)

Original x:  tensor([2., 2.]) Original y:  tensor([1.])
Transformed x_: tensor([300., 300.]) Transformed y_: tensor([200.])


### 3.2 Dataset 전체적용

In [0]:
compose_data_set = toy_set(transform = data_transform)

Let us print out the first 3 elements in different <code>toy_set</code> datasets in order to compare the output after different transforms have been applied: 

In [24]:
for i in range(3):
    print('Original')
    x, y = data_set[i]
    print( x, y)
    x_, y_ = cust_data_set[i]
    print('Transformed')
    print(x_, y_)
    print('Compose Transformed: ')
    x_co, y_co = compose_data_set[i]
    print(x_co ,y_co)
    print('-'*30)

Original
tensor([2., 2.]) tensor([1.])
Transformed
tensor([3., 3.]) tensor([2.])
Compose Transformed: 
tensor([300., 300.]) tensor([200.])
------------------------------
Original
tensor([2., 2.]) tensor([1.])
Transformed
tensor([3., 3.]) tensor([2.])
Compose Transformed: 
tensor([300., 300.]) tensor([200.])
------------------------------
Original
tensor([2., 2.]) tensor([1.])
Transformed
tensor([3., 3.]) tensor([2.])
Compose Transformed: 
tensor([300., 300.]) tensor([200.])
------------------------------
